In [ ]:
url = "https://ae.kickscrew.com/collections/nike"

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
driver = webdriver.Chrome()  # or webdriver.Firefox() or any other WebDriver

driver.get(url)

time.sleep(5)

search_results = driver.find_elements(By.CLASS_NAME,"ais-Hits-item")

data_list = []

for result in search_results:
    # Extract title, price, and image within each search result
    title = result.find_element(By.CLASS_NAME,"hit-image").get_attribute("alt")
    price = result.find_element(By.CLASS_NAME,"product-price").text
    image_url = result.find_element(By.CLASS_NAME,"hit-image").get_attribute("src")

    data_list.append([title, price, image_url])

# Close the browser window
driver.quit()

# Save data to CSV
csv_file_path = "scraped_data.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header row
    csv_writer.writerow(['Title', 'Price', 'Image URL'])

    # Write the data rows
    csv_writer.writerows(data_list)

print(f"Data has been saved to {csv_file_path}")

In [ ]:
data_list

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_product_title(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the HTML element containing the product title
        title_element = soup.find('div', class_='agl-js-product-title')

        # Extract the text content of the title element
        if title_element:
            product_title = title_element.text.strip()
            return product_title
        else:
            return "Title not found on the page."

    else:
        # Print an error message if the request was not successful
        print(f"Error: {response.status_code}")
        return None

# Example usage:
url = "https://ae.kickscrew.com/collections/nike"
title = get_product_title(url)

if title:
    print(f"The product title is: {title}")


In [ ]:
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.title

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
html = urlopen(BASE_URL)

In [ ]:
bs = BeautifulSoup(html.read(),'html.parser')

In [ ]:
bs.find_all('li',class_="ais-Hits-item")


In [ ]:
bs.h1

In [ ]:
bs.find_all('li')

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

# Use a webdriver to load the page (make sure to have the appropriate webdriver installed)
driver = webdriver.Chrome()
driver.get(BASE_URL)

# Get the page source after it's fully loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
bs = BeautifulSoup(page_source, 'html.parser')

# Find the element
result = bs.find(class_="ais-Hits-list")

# Close the webdriver
#driver.quit()


In [ ]:
from selenium.webdriver.common.by import By
driver.find_element(By.CLASS_NAME, 'agl-js-product-title').text

In [ ]:
driver.find_element(By.CLASS_NAME, 'hit-image').get_attribute('src')

In [ ]:
driver.find_element(By.CLASS_NAME, 'hit-image').get_attribute('alt')

In [ ]:
driver.find_element(By.CLASS_NAME, 'product-price').text

In [ ]:
result = bs.find_all('div', class_='agl-js-product-title')

In [ ]:
bs.get_attribute_list

In [ ]:
bs.h1

In [ ]:
driver.find_element(By.CLASS_NAME, 'agl-js-product-title')

In [ ]:
results_grid = driver.find_element(By.CLASS_NAME,"ais-Hits-list")

In [ ]:
[i.get_attribute('alt') for i in results_grid.find_elements(By.CLASS_NAME, 'hit-image')]

In [ ]:
[i.get_attribute('src') for i in results_grid.find_elements(By.CLASS_NAME, 'hit-image')]

In [ ]:
[i.text for i in results_grid.find_elements(By.CLASS_NAME, 'product-price')]

In [12]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_search_results(driver):
    search_results = driver.find_elements(By.CLASS_NAME, "ais-Hits-item")

    data_list = []

    for result in search_results:
        title = result.find_element(By.CLASS_NAME,"hit-image").get_attribute("alt")
        price = result.find_element(By.CLASS_NAME,"product-price").text
        image_url = result.find_element(By.CLASS_NAME,"hit-image").get_attribute("src")

        data_list.append([title, price, image_url])

    return data_list

url = "https://ae.kickscrew.com/collections/nike"
driver = webdriver.Chrome()

driver.get(url)

# Introduce a sleep to allow the page to load
time.sleep(5)

# Use WebDriverWait to wait for the presence of the first result
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ais-Hits-item")))

# Scrape data from the first page
all_data = scrape_search_results(driver)

# Check for pagination and navigate through pages
while True:
    try:
        # Look for the "Next Page" link
        next_link = driver.find_element(By.CSS_SELECTOR, ".ais-Pagination-item--nextPage")

        if "ais-Pagination-item--disabled" in next_link.get_attribute("class"):
            print("No more pages. Breaking out of the loop.")
            break  # Break out of the loop if the link is disabled

        # Click the "Next Page" link
        next_link.click()

        # Wait for the next page to load
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ais-Hits-item")))

        # Scrape data from the current page and append it to the existing data
        current_data = scrape_search_results(driver)
        all_data.extend(current_data)

        # Introduce a short sleep to avoid overwhelming the server with requests
        time.sleep(2)

    except Exception as e:
        print(f"Error during pagination: {e}")
        break  # Break out of the loop if there's an issue or if there's no "Next Page" button

# Close the browser window
driver.quit()

# Save data to CSV
csv_file_path = "scraped_data.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Title', 'Price', 'Image URL'])
    csv_writer.writerows(all_data)

print(f"Data has been saved to {csv_file_path}")


Error during pagination: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=103.0.5060.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5557151f3cd3 <unknown>
#1 0x555714ffb968 <unknown>
#2 0x555714ffe7c7 <unknown>
#3 0x555714ffe66b <unknown>
#4 0x555714ffe92c <unknown>
#5 0x555715032cfe <unknown>
#6 0x5557150331a1 <unknown>
#7 0x555715027b07 <unknown>
#8 0x555715050bdd <unknown>
#9 0x555715027a43 <unknown>
#10 0x555715050cbe <unknown>
#11 0x555715063ea8 <unknown>
#12 0x555715050aa3 <unknown>
#13 0x5557150263fa <unknown>
#14 0x555715027555 <unknown>
#15 0x55571523b2bd <unknown>
#16 0x55571523f418 <unknown>
#17 0x55571522536e <unknown>
#18 0x555715240078 <unknown>
#19 0x555715219bb0 <unknown>
#20 0x55571525cd58 <unknown>
#21 0x55571525ced8 <unknown>
#22 0x555715276cfd <unknown>
#23 0x7fbab6462609 <unknown>

Dat